In [1]:
import sys
sys.path.append('../')
from src import optimizer as _optimizer
from src import visualization as _visualization
from src import metrics as _metrics
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import seaborn as sns

sns.set(rc={'figure.figsize':(20,8)})
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
pd.set_option('display.max_rows', 120)

## Reading / Displaying the Data 

In [2]:
df_products = pd.read_csv('../data/products_features_exp4.csv')
df_products = df_products.set_index('code')
df_products.head()

,cat_code,source_channel_quantity_152bf0ce464047b9499ccb9e5b9b77a8,source_channel_quantity_3ab2427543039f8c9f17d06f6f65a3a7,source_channel_quantity_5a97b8efd901c1d28ff86522b95babb9,source_channel_quantity_67c19e107de33cab7ea9a9db8bc9ccd2,source_channel_quantity_7261d300057219056592010c7bdaf5ee,source_channel_quantity_98defd6ee70dfb1dea416cecdf391f58,source_channel_quantity_9d3e0fcbc1f16d80a76026e8f1c26002,source_channel_quantity_af082bb0c2fa1414655017d464aa0262,source_channel_quantity_ea2912716be1999ab62d5b9dfa4f58f9,...,month_quantity_9,month_quantity_10,month_quantity_11,month_quantity_12,perc_pis_cofins_quantity_0.0,perc_pis_cofins_quantity_9.25,weekend_quantity_0,weekend_quantity_1,kmeans_cluster,hc_cluster
code,,,,,,,,,,,,,,,,,,,,,
e6762ba2ffbca07ab6cee7551caeaad5,1,0.048780,0.0,0.000000,0.0,0.02439,0.048780,0.146341,0.000000,0.000000,...,0.02439,0.268293,0.073171,0.121951,0.000000,1.000000,0.756098,0.243902,4,9
d408e1b5e841dde4e15a4cfa182e3812,5,0.013333,0.0,0.026667,0.0,0.00000,0.040000,0.040000,0.000000,0.026667,...,0.00000,0.000000,0.186667,0.160000,0.000000,1.000000,0.746667,0.253333,8,6
29424aaf6e27a8dbe4b7273a0a39131d,5,0.034091,0.0,0.000000,0.0,0.00000,0.017045,0.051136,0.000000,0.051136,...,0.00000,0.000000,0.000000,0.005682,0.005682,0.994318,0.676136,0.323864,8,6
723f73c85e91fc31d147dfade389d4f9,0,0.034000,0.0,0.002000,0.0,0.03400,0.044000,0.078000,0.004000,0.018000,...,0.07000,0.082000,0.134000,0.074000,0.000000,1.000000,0.724000,0.276000,7,2
54209126056016c7c391c0c8fd8e6eff,0,0.013746,0.0,0.000000,0.0,0.00000,0.024055,0.079038,0.006873,0.020619,...,0.00000,0.000000,0.199313,0.226804,0.000000,1.000000,0.756014,0.243986,0,1


In [3]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130 entries, e6762ba2ffbca07ab6cee7551caeaad5 to 1c234775cae774823f38abe6721e61a4
Data columns (total 28 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   cat_code                                                  130 non-null    int64  
 1   source_channel_quantity_152bf0ce464047b9499ccb9e5b9b77a8  130 non-null    float64
 2   source_channel_quantity_3ab2427543039f8c9f17d06f6f65a3a7  130 non-null    float64
 3   source_channel_quantity_5a97b8efd901c1d28ff86522b95babb9  130 non-null    float64
 4   source_channel_quantity_67c19e107de33cab7ea9a9db8bc9ccd2  130 non-null    float64
 5   source_channel_quantity_7261d300057219056592010c7bdaf5ee  130 non-null    float64
 6   source_channel_quantity_98defd6ee70dfb1dea416cecdf391f58  130 non-null    float64
 7   source_channel_quantity_9d3e0fcbc1f16d80a76026e8f1c2

In [4]:
df_sales = pd.read_csv('../data/clean_step2.csv')
df_sales.head()

,order_id,code,quantity,price,pis_cofins,icms,tax_substitution,category,liquid_cost,order_status,...,perc_icms,perc_tax_substitution,unit_revenue,unit_price,unit_liquid_price,unit_perc_pis_cofins,unit_perc_icms,unit_perc_tax_substitution,unit_liquid_cost,unit_markup
0,bcb59c839e78b2601374cbad9239ca7b,e6762ba2ffbca07ab6cee7551caeaad5,1,978.90,90.5483,0.000,191.8416,4ece547755cba9e7fc14125bc895f31b,542.7065,entrega total,...,0.0,19.60,537.4868,978.90,1080.1933,9.25,0.0,19.60,542.7065,1.990382
1,88eb0ac86af1a521c0831298d22dea8b,e6762ba2ffbca07ab6cee7551caeaad5,1,978.90,90.5483,0.000,191.8416,4ece547755cba9e7fc14125bc895f31b,542.7065,entrega total,...,0.0,19.60,537.4868,978.90,1080.1933,9.25,0.0,19.60,542.7065,1.990382
2,1c175bc61b9b659bbf011b2e5e3dcec6,e6762ba2ffbca07ab6cee7551caeaad5,1,976.05,90.2846,0.000,192.3325,4ece547755cba9e7fc14125bc895f31b,542.7065,entrega total,...,0.0,19.71,535.3914,976.05,1078.0979,9.25,0.0,19.71,542.7065,1.986521
3,a8ad36828898fa3f6efeb5bd19c076f2,e6762ba2ffbca07ab6cee7551caeaad5,1,1089.10,100.7418,185.147,0.0000,4ece547755cba9e7fc14125bc895f31b,542.7065,entrega total,...,17.0,0.00,260.5047,1089.10,803.2112,9.25,17.0,0.00,542.7065,1.480010
4,9cbfaac6e04ea6ed454b843c94f2c29c,e6762ba2ffbca07ab6cee7551caeaad5,1,949.00,87.7825,170.820,0.0000,4ece547755cba9e7fc14125bc895f31b,542.7065,entrega total,...,18.0,0.00,147.6910,949.00,690.3975,9.25,18.0,0.00,542.7065,1.272138


In [5]:
df_sales.columns

Index(['order_id', 'code', 'quantity', 'price', 'pis_cofins', 'icms',
       'tax_substitution', 'category', 'liquid_cost', 'order_status',
       'capture_date', 'process_date', 'process_status', 'source_channel',
       'liquid_price', 'revenue', 'markup', 'week_number', 'month',
       'perc_pis_cofins', 'perc_icms', 'perc_tax_substitution', 'unit_revenue',
       'unit_price', 'unit_liquid_price', 'unit_perc_pis_cofins',
       'unit_perc_icms', 'unit_perc_tax_substitution', 'unit_liquid_cost',
       'unit_markup'],
      dtype='object')

In [6]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144731 entries, 0 to 144730
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   order_id                    144731 non-null  object 
 1   code                        144731 non-null  object 
 2   quantity                    144731 non-null  int64  
 3   price                       144731 non-null  float64
 4   pis_cofins                  144731 non-null  float64
 5   icms                        144731 non-null  float64
 6   tax_substitution            144731 non-null  float64
 7   category                    144731 non-null  object 
 8   liquid_cost                 144731 non-null  float64
 9   order_status                144731 non-null  object 
 10  capture_date                144731 non-null  object 
 11  process_date                144731 non-null  object 
 12  process_status              144731 non-null  object 
 13  source_channel

## Incorporating some feature insights from previous EDAs

* perc_pis_cofins - Helps to understand the type of product. 
* source_channel - indicate some movement, there are 5 top channels that might help creating diversity (5 times more instances to predict. 
* hc_cluster and kmeans_cluster, will act as an embedding of seasonal and source behaviours. 
* markup/revenue, I will use the markup since it is a function of the liquid cost and reflect the revenue. Both a are f(liquid_cost) and quantity = g(f(liquid_cost)) ==> demand = g(price) 
* sum(quantity) - will be the Y 
* I'll also make a quantization of the markup to guarantee a reasoable cardinality 


In [7]:
df_sales['unit_markup'] = df_sales['unit_markup'].apply(lambda x: np.round(x, 1))

In [8]:
df_sales['year_month'] = df_sales['capture_date'].apply(lambda x: x[:7])

In [9]:
df_dataset = df_sales.groupby(['code', 'year_month', 'month', 'perc_pis_cofins', 'source_channel', 'unit_markup']).agg({'quantity':'sum'})

In [10]:
df_dataset = df_dataset.join(df_products[['hc_cluster', 'kmeans_cluster','cat_code']], on='code', how='left')

In [11]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 28761 entries, ('0671c2b9132a3f5215a4212ce0691694', '2016-11', 11, 0.0, 'a578e71c3216f513a84ec6a46084fd3a', 1.7) to ('ffdad3ddbaf6c76c9bba1b48c51e03c6', '2017-06', 6, 9.25, 'fc7020775a7cdf161ab5267985c54601', 1.9)
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   quantity        28761 non-null  int64
 1   hc_cluster      28761 non-null  int64
 2   kmeans_cluster  28761 non-null  int64
 3   cat_code        28761 non-null  int64
dtypes: int64(4)
memory usage: 1.1+ MB


In [12]:
df_dataset = df_dataset.reset_index()
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28761 entries, 0 to 28760
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   code             28761 non-null  object 
 1   year_month       28761 non-null  object 
 2   month            28761 non-null  int64  
 3   perc_pis_cofins  28761 non-null  float64
 4   source_channel   28761 non-null  object 
 5   unit_markup      28761 non-null  float64
 6   quantity         28761 non-null  int64  
 7   hc_cluster       28761 non-null  int64  
 8   kmeans_cluster   28761 non-null  int64  
 9   cat_code         28761 non-null  int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 2.2+ MB


## Checking cardinalities
The idea is to keep those at some levels during the train, validation and test separations

In [13]:
df_dataset.groupby(['code']).source_channel.nunique().reset_index().describe()

,source_channel
count,130.000000
mean,9.169231
std,2.255531
min,1.000000
25%,8.000000
50%,9.000000
75%,11.000000
max,14.000000


In [14]:
df_dataset.perc_pis_cofins.unique()

array([0.  , 9.25])

In [15]:
df_dataset.groupby(['code']).perc_pis_cofins.nunique().reset_index().describe()

,perc_pis_cofins
count,130.000000
mean,1.607692
std,0.490153
min,1.000000
25%,1.000000
50%,2.000000
75%,2.000000
max,2.000000


In [16]:
df_dataset.groupby(['code']).unit_markup.nunique().reset_index().describe()

,unit_markup
count,130.000000
mean,14.400000
std,3.738756
min,1.000000
25%,13.000000
50%,14.000000
75%,16.000000
max,24.000000


In [17]:
df_dataset.groupby(['code']).month.nunique().reset_index().describe()

,month
count,130.000000
mean,10.276923
std,2.369101
min,1.000000
25%,9.000000
50%,11.000000
75%,12.000000
max,12.000000


In [18]:
df_dataset.describe()

,month,perc_pis_cofins,unit_markup,quantity,hc_cluster,kmeans_cluster,cat_code
count,28761.000000,28761.000000,28761.000000,28761.000000,28761.000000,28761.000000,28761.000000
mean,6.209311,8.813245,1.629989,5.287090,2.133514,4.236397,0.586454
std,3.630978,1.961978,0.389422,13.586242,1.614470,3.279354,1.763417
min,1.000000,0.000000,0.500000,1.000000,1.000000,0.000000,0.000000
25%,3.000000,9.250000,1.300000,1.000000,1.000000,0.000000,0.000000
50%,5.000000,9.250000,1.600000,2.000000,2.000000,7.000000,0.000000
75%,10.000000,9.250000,1.900000,4.000000,2.000000,7.000000,0.000000
max,12.000000,9.250000,4.800000,364.000000,9.000000,8.000000,10.000000


In [19]:
df_dist = df_dataset[['code', 'month', 'unit_markup']].groupby(['code', 'month']).count().reset_index()
df_dist.columns = ['code', 'month', 'instace_count']

In [20]:
df_dist[df_dist.instace_count < 2].sort_values('code')

,code,month,instace_count
139,1c619f92929dbaa41df54608dc70a7ce,7,1
149,1dbe25b2fd344aed0c444fd6b715525b,6,1
161,1f12f1e1b9e7a20d4ad9dd549ec072c0,10,1
164,213005fe5d815c539812614f1f6a7768,1,1
193,24549e47832f72f647d40a86b43b6925,11,1
200,29424aaf6e27a8dbe4b7273a0a39131d,12,1
225,2bc9e1d807d8f9187b8650acd35a14e5,2,1
226,2bc9e1d807d8f9187b8650acd35a14e5,5,1
292,373cc1cfc10a45488be6b97bd5e94c44,6,1
293,373cc1cfc10a45488be6b97bd5e94c44,7,1


In [21]:
len(df_dist[df_dist.instace_count < 2].code.unique())

40

In [22]:
df_dataset['Y'] = df_dataset.quantity
df_dataset = df_dataset.drop(columns='quantity')

## Converting to dummies the categorical columns

In [23]:
cat_columns = ['source_channel', 'hc_cluster', 'kmeans_cluster', 'perc_pis_cofins', 'cat_code']
for col in cat_columns:     
    df_dataset[col] = df_dataset[col].astype('category')

In [24]:
df_dataset_v2 = pd.get_dummies(df_dataset, columns=cat_columns)

In [25]:
df_dataset_v2.columns

Index(['code', 'year_month', 'month', 'unit_markup', 'Y',
       'source_channel_152bf0ce464047b9499ccb9e5b9b77a8',
       'source_channel_2934a86a91bfa55d7f20b4f08a441fac',
       'source_channel_3ab2427543039f8c9f17d06f6f65a3a7',
       'source_channel_5a97b8efd901c1d28ff86522b95babb9',
       'source_channel_67c19e107de33cab7ea9a9db8bc9ccd2',
       'source_channel_7261d300057219056592010c7bdaf5ee',
       'source_channel_98defd6ee70dfb1dea416cecdf391f58',
       'source_channel_9d3e0fcbc1f16d80a76026e8f1c26002',
       'source_channel_a578e71c3216f513a84ec6a46084fd3a',
       'source_channel_af082bb0c2fa1414655017d464aa0262',
       'source_channel_b76eb9b8fc0f17098812da9117d3e500',
       'source_channel_e9b49f9086ba813ca3f0b321710fef16',
       'source_channel_ea2912716be1999ab62d5b9dfa4f58f9',
       'source_channel_fc7020775a7cdf161ab5267985c54601', 'hc_cluster_1',
       'hc_cluster_2', 'hc_cluster_3', 'hc_cluster_4', 'hc_cluster_5',
       'hc_cluster_6', 'hc_cluster_7', 'hc_

In [33]:
df_dataset_v2.head()

,code,year_month,month,unit_markup,Y,source_channel_152bf0ce464047b9499ccb9e5b9b77a8,source_channel_2934a86a91bfa55d7f20b4f08a441fac,source_channel_3ab2427543039f8c9f17d06f6f65a3a7,source_channel_5a97b8efd901c1d28ff86522b95babb9,source_channel_67c19e107de33cab7ea9a9db8bc9ccd2,...,cat_code_1,cat_code_2,cat_code_3,cat_code_4,cat_code_5,cat_code_6,cat_code_7,cat_code_8,cat_code_9,cat_code_10
0,0671c2b9132a3f5215a4212ce0691694,2016-11,11,1.7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0671c2b9132a3f5215a4212ce0691694,2016-11,11,1.1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0671c2b9132a3f5215a4212ce0691694,2016-11,11,1.2,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0671c2b9132a3f5215a4212ce0691694,2016-11,11,1.4,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0671c2b9132a3f5215a4212ce0691694,2016-11,11,1.5,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df_dataset_v2.to_csv('../data/forecast_dataset_v2.csv', index=False)

## Splitting the Sets
* We will use the temporal series cross validation
* 3 months minimum intervals
* 3 training, 6 validation, 3 test
* 4 training, 5 validation, 3 test
* 5 training, 4 validation, 3 test
* 6 training, 3 validation, 3 test

In [27]:
df_sets = df_dataset_v2[['year_month', 'month']].drop_duplicates().sort_values(['year_month', 'month'])

In [28]:
test_months = df_sets['year_month'].tail(4).values[:3]

In [29]:
test_months

array(['2017-03', '2017-04', '2017-05'], dtype=object)

In [30]:
train_val = []
for i in range(7): 
    sets_year_month = list(df_sets['year_month'].head(9).values)
    train_val.append({
        'set': i,
        'train': sets_year_month[:3+i], 
        'val'  : test_months if i ==6 else sets_year_month[3+i:], 
    })
    print(train_val[-1])

{'set': 0, 'train': ['2016-06', '2016-07', '2016-08'], 'val': ['2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02']}
{'set': 1, 'train': ['2016-06', '2016-07', '2016-08', '2016-09'], 'val': ['2016-10', '2016-11', '2016-12', '2017-01', '2017-02']}
{'set': 2, 'train': ['2016-06', '2016-07', '2016-08', '2016-09', '2016-10'], 'val': ['2016-11', '2016-12', '2017-01', '2017-02']}
{'set': 3, 'train': ['2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11'], 'val': ['2016-12', '2017-01', '2017-02']}
{'set': 4, 'train': ['2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12'], 'val': ['2017-01', '2017-02']}
{'set': 5, 'train': ['2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-01'], 'val': ['2017-02']}
{'set': 6, 'train': ['2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02'], 'val': array(['2017-03', '2017-04', '2017-05'], dtype=object)}


In [31]:
df_train_val_test = pd.DataFrame(train_val)
df_train_val_test['reserved'] = df_train_val_test.set == 6

In [32]:
df_train_val_test.to_csv('../data/train_val_test.csv', index=False)